In [1]:
from dotenv import load_dotenv
import os, pickle, asyncio
load_dotenv(".env")
OPENAI_KEY = os.environ.get("OPENAI_KEY")
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

In [73]:
file = open("instructions.txt", "r")
instructions = file.read()
file = open("test.txt", "r")
abstracts = file.read()

In [4]:
#Preprocessing
import pandas as pd

data = []

file = open("abstract-alzheimers-set.txt", "r")
raw = file.read()

#split into publications
publications = raw.split("\n\n\n")
print(len(publications))

for pub in publications:
    #split into elements
    element_list = pub.split("\n\n")

    #get title 
    title = element_list[1]

    #get abstract
    element_list = [item for item in element_list if "Author information:" not in item]
    abstract = max(element_list, key=len)

    #get doi
    identifier = next((item for item in element_list if "DOI:" in item), None)

    #save data in list
    data.append([title, abstract, identifier])
    
# Convert list to DataFrame
publications_df = pd.DataFrame(data, columns=['Title', 'Abstract', 'Identifier'])
publications_df.head()


#publications_df["Abstract"][0]

10


,Title,Abstract,Identifier
0,Alzheimer's disease.,1. Nature. 2011 Jul 13;475(7355):S1. doi: 10.1...,DOI: 10.1038/475S1a\nPMID: 21760574 [Indexed f...
1,Alzheimer's disease.,"In this Seminar, we highlight the main develop...",DOI: 10.1016/S0140-6736(20)32205-4\nPMCID: PMC...
2,Alzheimer's disease.,Alzheimer's disease (AD) dementia refers to a ...,DOI: 10.1016/B978-0-12-804766-8.00013-3\nPMID:...
3,Current understanding of Alzheimer's disease d...,Alzheimer's disease is the most common cause o...,DOI: 10.12688/f1000research.14506.1\nPMCID: PM...
4,Alzheimer's Disease-Biochemical and Psychologi...,There is a paucity of empirical research on th...,DOI: 10.3390/ijms24021059\nPMCID: PMC9866942\n...


In [44]:
def get_category (instructions, abstracts):

   response = client.chat.completions.create(
   model="gpt-4-1106-preview" #"gpt-3.5-turbo-1106",
   #temperature=0.5,
  # response_format={ "type": "json_object" },
   messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": abstracts}
    ]
   )
   return response.choices[0].message.content



In [74]:
with open("output.txt", "w") as file:
	content = get_category (instructions, abstracts)
	file.write(content)
	file.close()